# 3D Cartpole Robot 🤖 🐬
In this colab i put up a simulated 3D cartpole. The cart can slide freely on the ground plane without any friction; the pole is free to rotate in the plane above the cart.

##Burocrazia 🔖 🐧
1. ⚙ ⚙ Installing `pybullet`
2. 📡 setting uo the connection with the engine

  🛑 *Colab does not support the `p.GUI` mode, so the only option is `p.DIRECT`: no big deal, u just can render the thing in real time*

3. 🌐 Setting up the ground plane

In [9]:
!pip install pybullet

In [29]:
import pybullet as p
import pybullet_data
import numpy as np

# Connect to PyBullet
p.connect(p.DIRECT)

# Set additional search path for PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load a plane for the simulation
p.loadURDF("plane.urdf")


0

## Oggettini 🪄 🐏
Declaring parameters for the object that i want and creating them in the space i just set up. The parameters are that of the standard cartole problem.

*   🦘 🕋   Declaring a *cart*: a box of shape `[x,y,z] =  [0.5, 0.5, 0.2]`. I put it just above the ground plane 🛸 so that we can ignore the friction.

*   🐘 🪐   Declaring a *pole* attached to the cart: a cilynder of shape `[x,y,z]  =  [0.5, 0.5, 0.1]`

*   ⚙ 🏯 🦊 Add *joints* between the cart and the pole. The pole can rotate in a plane and not just on one axis, so we have to declare a spherical joint





🛑 *Spherical joints are not easy to limitate, check double before code. However, for now, it's the easiest solution for the rotation in all of the plane.*

In [30]:
# Define cartpole parameters
mass_cart = 1
cart_size = [0.5, 0.5, 0.2]
mass_pole = 0.1
pole_size = [0.05, 0.05, 1.0]
cart_position = [0, 0, 0.1]  # Slightly above the ground
pole_position = [0, 0, 0.5]  # on the center of the cart

# Create the cart (base)
cart_collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[s / 2 for s in cart_size])

print(cart_collision_shape)
cart_visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=[s / 2 for s in cart_size], rgbaColor=[0, 1, 1, 1])
print(cart_visual_shape)
# Create the pole (link)
pole_collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[s / 2 for s in pole_size])
print(pole_collision_shape)
pole_visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=[s / 2 for s in pole_size], rgbaColor=[1, 0.5, 0, 1])
print(pole_visual_shape)

# Center of mass for the pole will be halfway along its length
pole_inertial_frame_position = [0, 0, -pole_size[2] / 2]  # Center of mass is halfway along the pole

# Create the cart-pole system with a spherical joint for full rotation
# Create the cart-pole system with a spherical joint for full rotation
cartpole_id = p.createMultiBody(
    baseMass=mass_cart,
    baseCollisionShapeIndex=cart_collision_shape,
    baseVisualShapeIndex=cart_visual_shape,
    basePosition=cart_position,
    baseInertialFramePosition=[0, 0, 0],  # Base Inertia (no offset needed for the cart)

    # Link 1: Pole with spherical joint
    linkMasses=[mass_pole],  # Pole's mass
    linkCollisionShapeIndices=[pole_collision_shape],  # Collision shape of the pole
    linkVisualShapeIndices=[pole_visual_shape],  # Visual shape of the pole
    linkPositions=[[0, 0, cart_size[2] / 2]],  # Pole position relative to the cart
    linkOrientations=[[0, 0, 0, 1]],  # Orientation of the pole
    linkInertialFramePositions=[pole_inertial_frame_position],  # Inertia offset for the pole
    linkInertialFrameOrientations=[[0, 0, 0, 1]],  # Inertial orientation of the pole
    linkParentIndices=[0],  # Linked to the cart (base index = 0)
    linkJointTypes=[p.JOINT_SPHERICAL],  # Spherical joint for full 3D rotation
    linkJointAxis=[[0, 0, 0]]  # Spherical joint doesn't need an axis definition
)




0
0
1
1


##useful information and how to retrieve them 🔖 🪃

In [35]:
# Get the number of joints
num_joints = p.getNumJoints(cartpole_id)
print(f"Number of joints: {num_joints}")

# Get information about each joint
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(cartpole_id, joint_index)
    joint_name = joint_info[1].decode('UTF-8')  # Decode bytes to string
    joint_type = joint_info[2]  # Joint type (e.g., revolute, spherical)

    print(f"Joint Index: {joint_index}, Name: {joint_name}, Type: {joint_type}")

#reset joint state to the position and velocity i give him
p.resetJointState(cartpole_id, 0,  0.1, 0.3)  # Example values

# Get the position and orientation of the cart (base)
cart_position, cart_orientation = p.getBasePositionAndOrientation(cartpole_id)

# Get the velocity (linear and angular) of the cart (base)
cart_linear_velocity, cart_angular_velocity = p.getBaseVelocity(cartpole_id)

# Get the position, orientation, and velocity (linear and angular) of the pole (link 1)
pole_link_state = p.getLinkState(cartpole_id, 0, computeLinkVelocity=True)
pole_position = pole_link_state[0]  # Position of the pole
pole_orientation = pole_link_state[1]  # Orientation of the pole
pole_linear_velocity = pole_link_state[6]  # Linear velocity of the pole
pole_angular_velocity = pole_link_state[7]  # Angular velocity of the pole

# Print the positions and velocities of the cart and the pole
print("Cart Position:", cart_position)
print("Cart Orientation (Quaternion):", cart_orientation)
print("Cart Linear Velocity:", cart_linear_velocity)
print("Cart Angular Velocity:", cart_angular_velocity)
print("Pole Position:", pole_position)
print("Pole Orientation (Quaternion):", pole_orientation)
print("Pole Linear Velocity:", pole_linear_velocity)
print("Pole Angular Velocity:", pole_angular_velocity)
print()

print(cart_position[0:2])

Number of joints: 1
Joint Index: 0, Name: joint1, Type: 2
Cart Position: (0.0, 0.0, 0.1)
Cart Orientation (Quaternion): (0.0, 0.0, 0.0, 1.0)
Cart Linear Velocity: (0.0, 0.0, 0.0)
Cart Angular Velocity: (0.0, 0.0, 0.0)
Pole Position: (0.0, 0.0, -0.30000000000000004)
Pole Orientation (Quaternion): (0.0, 0.0, 0.0, 1.0)
Pole Linear Velocity: (0.0, 0.0, 0.0)
Pole Angular Velocity: (0.0, 0.0, 0.0)

(0.0, 0.0)


## Start simulation 🚀 🪖
Adding gravity 🍎 ⬇ to the system and starting the physics simulation 🏃 🔥 🛕 ⛩ ⛪ .

In the simulation i retrieve the state of the cart (base) and the pole (link) trough the function `getBase...` and `getLink...`


In [27]:
# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

p.resetSimulation()

randstate_position = np.random.uniform(low=-0.05, high=0.05, size=(3,))
print(randstate_position)
randstate_velocity = np.random.uniform(low=-0.05, high=0.05, size=(3,))
# Resetting the joint state for the cart (base)
p.resetJointState(cartpole_id, 0,  0.1, 0.3)  # Example values

# Resetting the joint state for the pole (assuming it's index 1)
p.resetJointState(cartpole_id, 1, 0.2, 0.4)

'''
# Run the simulation and track the position, orientation, and velocity
for step in range(10):
    p.stepSimulation()

    # Get the position and orientation of the cart (base)
    cart_position, cart_orientation = p.getBasePositionAndOrientation(cartpole_id)

    # Get the velocity (linear and angular) of the cart (base)
    cart_linear_velocity, cart_angular_velocity = p.getBaseVelocity(cartpole_id)

    # Get the position, orientation, and velocity (linear and angular) of the pole (link 1)
    pole_link_state = p.getLinkState(cartpole_id, 0, computeLinkVelocity=True)
    pole_position = pole_link_state[0]  # Position of the pole
    pole_orientation = pole_link_state[1]  # Orientation of the pole
    pole_linear_velocity = pole_link_state[6]  # Linear velocity of the pole
    pole_angular_velocity = pole_link_state[7]  # Angular velocity of the pole

    # Print the positions and velocities of the cart and the pole
    print("Cart Position:", cart_position)
    print("Cart Orientation (Quaternion):", cart_orientation)
    print("Cart Linear Velocity:", cart_linear_velocity)
    print("Cart Angular Velocity:", cart_angular_velocity)
    print("Pole Position:", pole_position)
    print("Pole Orientation (Quaternion):", pole_orientation)
    print("Pole Linear Velocity:", pole_linear_velocity)
    print("Pole Angular Velocity:", pole_angular_velocity)
    print()
'''
p.disconnect()


0
[-0.00713148 -0.04438448  0.03860238]


error: Joint index out-of-range.